In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.utils import to_categorical
np.random.seed(178)

In [2]:
csv_file_names = ['Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',
                  'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv',
                  'Friday-WorkingHours-Morning.pcap_ISCX.csv',
                  'Monday-WorkingHours.pcap_ISCX.csv',
                  'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
                  'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
                  'Tuesday-WorkingHours.pcap_ISCX.csv',
                  'Wednesday-workingHours.pcap_ISCX.csv']
dfs = []
for file_name in csv_file_names:
    df = pd.read_csv(file_name)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

In [3]:
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830738,53,32215,4,2,112,152,28,28,28.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830739,53,324,2,2,84,362,42,42,42.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830740,58030,82,2,1,31,6,31,0,15.5,21.92031,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830741,53,1048635,6,2,192,256,32,32,32.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [4]:
df[' Label'].isnull().sum()

0

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max         

In [6]:
df.isnull().sum()

 Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
 Label                         0
Length: 79, dtype: int64

In [7]:
from itertools import combinations

def data_cleaning(df):
    df.columns=df.columns.str.strip()
    print("Dataset Shape: ",df.shape)
    
    num=df._get_numeric_data()
    num[num<0]=0
    
    zero_variance_cols=[]
    for col in df.columns:
        if len(df[col].unique()) == 1:
            zero_variance_cols.append(col)
    df.drop(columns=zero_variance_cols,axis=1,inplace=True)
    print("Zero Variance Columns: ",zero_variance_cols, " are dropped!!")
    print("Shape after removing the zero variance columns: ",df.shape)
    
    df.replace([np.inf,-np.inf],np.nan,inplace=True)
    print(df.isna().any(axis=1).sum(), "rows dropped")
    df.dropna(inplace=True)
    print("Shape after Removing NaN: ",df.shape)
    
    df.drop_duplicates(inplace=True)
    print("Shape after dropping duplicates: ",df.shape)
    
    column_pairs = [(i,j) for i,j in combinations(df,2) if df[i].equals(df[j])]
    ide_cols=[]
    for col_pair in column_pairs:
        ide_cols.append(col_pair[1])
    df.drop(columns=ide_cols,axis=1,inplace=True)
    print("Columns which have identical values: ",column_pairs," dropped!")
    print("Shape after removing identical value columns: ",df.shape)
    return df
df=data_cleaning(df)

Dataset Shape:  (2830743, 79)
Zero Variance Columns:  ['Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']  are dropped!!
Shape after removing the zero variance columns:  (2830743, 71)
2867 rows dropped
Shape after Removing NaN:  (2827876, 71)
Shape after dropping duplicates:  (2520798, 71)
Columns which have identical values:  [('Total Fwd Packets', 'Subflow Fwd Packets'), ('Total Backward Packets', 'Subflow Bwd Packets'), ('Fwd PSH Flags', 'SYN Flag Count'), ('Fwd URG Flags', 'CWE Flag Count'), ('Fwd Header Length', 'Fwd Header Length.1')]  dropped!
Shape after removing identical value columns:  (2520798, 66)


In [8]:
df.columns

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags',
       'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s',
       'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length',
       'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
       'FIN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',

In [9]:
df['Label'].value_counts()

BENIGN                        2095057
DoS Hulk                       172846
DDoS                           128014
PortScan                        90694
DoS GoldenEye                   10286
FTP-Patator                      5931
DoS slowloris                    5385
DoS Slowhttptest                 5228
SSH-Patator                      3219
Bot                              1948
Web Attack � Brute Force         1470
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: Label, dtype: int64

In [10]:
def change_label(df):
  df.Label.replace(['DoS Hulk','DDoS','DoS GoldenEye','DoS slowloris','DoS Slowhttptest','Heartbleed'],'Dos',inplace=True)
  df.Label.replace(['Web Attack � Brute Force','Web Attack � XSS','Web Attack � Sql Injection'],'WebAttack',inplace=True)      
  df.Label.replace(['FTP-Patator','SSH-Patator'],'BruteForce',inplace=True)

In [11]:
change_label(df)

In [12]:
df['Label'].value_counts()

BENIGN          2095057
Dos              321770
PortScan          90694
BruteForce         9150
WebAttack          2143
Bot                1948
Infiltration         36
Name: Label, dtype: int64

In [13]:
Label = pd.DataFrame(df.Label)
Label

,Label
0,BENIGN
1,BENIGN
2,BENIGN
3,BENIGN
4,BENIGN
...,...
2830738,BENIGN
2830739,BENIGN
2830740,BENIGN
2830741,BENIGN


In [14]:
std_scaler = MinMaxScaler()
def standardization(df,col):
    for i in col:
        arr = df[i]
        arr = np.array(arr)
        df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    return df

numeric_col = df.select_dtypes(include='number').columns
df = standardization(df,numeric_col)

In [15]:
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.837186,2.500000e-08,0.000005,0.000000,9.302326e-07,0.000000e+00,0.000242,0.002581,0.001010,0.000000,...,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,0.840070,9.083333e-07,0.000000,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.001010,0.000000,...,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,0.840085,4.333333e-07,0.000000,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.001010,0.000000,...,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,0.705516,2.833333e-07,0.000000,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.001010,0.000000,...,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,0.837156,2.500000e-08,0.000005,0.000000,9.302326e-07,0.000000e+00,0.000242,0.002581,0.001010,0.000000,...,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830738,0.000809,2.684583e-04,0.000014,0.000007,8.682171e-06,2.319007e-07,0.001128,0.012043,0.004713,0.000000,...,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2830739,0.000809,2.700000e-06,0.000005,0.000007,6.511628e-06,5.522898e-07,0.001692,0.018065,0.007070,0.000000,...,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2830740,0.885481,6.833333e-07,0.000005,0.000003,2.403101e-06,9.153974e-09,0.001249,0.000000,0.002609,0.003076,...,0.231884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2830741,0.000809,8.738625e-03,0.000023,0.000007,1.488372e-05,3.905696e-07,0.001289,0.013763,0.005386,0.000000,...,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2520798 entries, 0 to 2830742
Data columns (total 66 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Destination Port             float64
 1   Flow Duration                float64
 2   Total Fwd Packets            float64
 3   Total Backward Packets       float64
 4   Total Length of Fwd Packets  float64
 5   Total Length of Bwd Packets  float64
 6   Fwd Packet Length Max        float64
 7   Fwd Packet Length Min        float64
 8   Fwd Packet Length Mean       float64
 9   Fwd Packet Length Std        float64
 10  Bwd Packet Length Max        float64
 11  Bwd Packet Length Min        float64
 12  Bwd Packet Length Mean       float64
 13  Bwd Packet Length Std        float64
 14  Flow Bytes/s                 float64
 15  Flow Packets/s               float64
 16  Flow IAT Mean                float64
 17  Flow IAT Std                 float64
 18  Flow IAT Max                 float64
 19  

In [17]:
X = df.drop('Label', axis=1)
y = df['Label']

In [18]:
# One Hot encoding  (BENIGN, Bot, BruteForce, DoS, Infiltration, PortScan, WebAttack)

encoder = OneHotEncoder(sparse=False)
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))
y_encoded = pd.DataFrame(y_encoded)
y_encoded

C:\Users\aryan\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,0,1,2,3,4,5,6
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2520793,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2520794,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2520795,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2520796,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
X = X.reset_index(drop=True)

In [22]:
X

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.837186,2.500000e-08,0.000005,0.000000,9.302326e-07,0.000000e+00,0.000242,0.002581,0.001010,0.000000,...,0.000005,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.840070,9.083333e-07,0.000000,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.001010,0.000000,...,0.000000,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.840085,4.333333e-07,0.000000,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.001010,0.000000,...,0.000000,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.705516,2.833333e-07,0.000000,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.001010,0.000000,...,0.000000,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.837156,2.500000e-08,0.000005,0.000000,9.302326e-07,0.000000e+00,0.000242,0.002581,0.001010,0.000000,...,0.000005,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2520793,0.000809,2.684583e-04,0.000014,0.000007,8.682171e-06,2.319007e-07,0.001128,0.012043,0.004713,0.000000,...,0.000014,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2520794,0.000809,2.700000e-06,0.000005,0.000007,6.511628e-06,5.522898e-07,0.001692,0.018065,0.007070,0.000000,...,0.000005,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2520795,0.885481,6.833333e-07,0.000005,0.000003,2.403101e-06,9.153974e-09,0.001249,0.000000,0.002609,0.003076,...,0.000000,0.231884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2520796,0.000809,8.738625e-03,0.000023,0.000007,1.488372e-05,3.905696e-07,0.001289,0.013763,0.005386,0.000000,...,0.000023,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
df = pd.concat([X, y_encoded], axis=1)
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Idle Std,Idle Max,Idle Min,0,1,2,3,4,5,6
0,0.837186,2.500000e-08,0.000005,0.000000,9.302326e-07,0.000000e+00,0.000242,0.002581,0.001010,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.840070,9.083333e-07,0.000000,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.001010,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.840085,4.333333e-07,0.000000,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.001010,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.705516,2.833333e-07,0.000000,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.001010,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.837156,2.500000e-08,0.000005,0.000000,9.302326e-07,0.000000e+00,0.000242,0.002581,0.001010,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2520793,0.000809,2.684583e-04,0.000014,0.000007,8.682171e-06,2.319007e-07,0.001128,0.012043,0.004713,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2520794,0.000809,2.700000e-06,0.000005,0.000007,6.511628e-06,5.522898e-07,0.001692,0.018065,0.007070,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2520795,0.885481,6.833333e-07,0.000005,0.000003,2.403101e-06,9.153974e-09,0.001249,0.000000,0.002609,0.003076,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2520796,0.000809,8.738625e-03,0.000023,0.000007,1.488372e-05,3.905696e-07,0.001289,0.013763,0.005386,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2520798 entries, 0 to 2520797
Data columns (total 72 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Destination Port             float64
 1   Flow Duration                float64
 2   Total Fwd Packets            float64
 3   Total Backward Packets       float64
 4   Total Length of Fwd Packets  float64
 5   Total Length of Bwd Packets  float64
 6   Fwd Packet Length Max        float64
 7   Fwd Packet Length Min        float64
 8   Fwd Packet Length Mean       float64
 9   Fwd Packet Length Std        float64
 10  Bwd Packet Length Max        float64
 11  Bwd Packet Length Min        float64
 12  Bwd Packet Length Mean       float64
 13  Bwd Packet Length Std        float64
 14  Flow Bytes/s                 float64
 15  Flow Packets/s               float64
 16  Flow IAT Mean                float64
 17  Flow IAT Std                 float64
 18  Flow IAT Max                 float64
 19  

In [24]:
X_train,X_test,y_train,y_test=train_test_split(X,y_encoded,test_size=0.2)
print(X_train.shape," ",X_test.shape)
print(y_train.shape," ",y_test.shape)

(2016638, 65)   (504160, 65)
(2016638, 7)   (504160, 7)


In [26]:
#selecting 1% of random rows for better running time

X_train = X_train.sample(frac=0.1, replace=True, random_state=1)
y_train = y_train.sample(frac=0.1, replace=True, random_state=1)
X_test = X_test.sample(frac=0.1, replace=True, random_state=1)
y_test = y_test.sample(frac=0.1, replace=True, random_state=1)
print (X_train.shape, y_train.shape)
print( X_test.shape, y_test.shape)

(201664, 65) (201664, 7)
(50416, 65) (50416, 7)


In [27]:
sel = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1))
sel.fit(X_train, y_train)
sel.get_support()

X_train.columns

features = X_train.columns[sel.get_support()]
print(features)

print(len(features))

np.mean(sel.estimator_.feature_importances_)
print(sel.estimator_.feature_importances_)

X_train_rfe = sel.transform(X_train)
X_test_rfe = sel.transform(X_test)

Index(['Destination Port', 'Total Fwd Packets', 'Total Length of Fwd Packets',
       'Bwd Packet Length Max', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow IAT Max', 'Fwd IAT Std',
       'Fwd Header Length', 'Bwd Header Length', 'Max Packet Length',
       'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
       'PSH Flag Count', 'Average Packet Size', 'Avg Bwd Segment Size',
       'Subflow Fwd Bytes', 'act_data_pkt_fwd'],
      dtype='object')
19
[2.62223883e-02 6.84404594e-03 2.51199721e-02 9.39040121e-03
 2.31066177e-02 7.15326822e-03 1.28230971e-02 3.70566242e-03
 1.03270815e-02 3.59877884e-03 3.55578848e-02 6.72993025e-03
 5.12465985e-02 1.12368498e-01 1.01692584e-02 3.73427589e-03
 4.80745026e-03 7.90369184e-03 2.28617928e-02 2.22072666e-03
 1.17374855e-02 1.01216111e-02 2.24947678e-02 1.12859915e-02
 6.96899861e-03 1.37132277e-03 2.01034759e-03 1.38901592e-03
 2.71573480e-03 1.61371918e-03 8.76495062e-04 8.23001531e-09
 2.42208709e-02 1.65

In [28]:
X_train_rfe=np.array(X_train_rfe)
X_test_rfe=np.array(X_test_rfe)
y=np.array(y)

In [30]:
X_train = np.reshape(X_train_rfe, ( X_train_rfe.shape[0], 1 , X_train_rfe.shape[1] ))
X_test = np.reshape(X_test_rfe, ( X_test_rfe.shape[0], 1,  X_test_rfe.shape[1] ))

In [37]:
from keras.layers import Dense, LSTM, MaxPool1D, Flatten, Dropout # importing dense layer
from keras.models import Sequential #importing Sequential layer
from keras.layers import Input
from keras.models import Model

model = Sequential() # initializing model
model.add(LSTM(64,return_sequences=True,input_shape = (1, X_train_rfe.shape[1])))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True))
model.add(Flatten())
model.add(Dense(units=50))
# output layer with softmax activation
model.add(Dense(units=7,activation='softmax'))

In [38]:
# defining loss function, optimizer, metrics and then compiling model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
# summary of model layers
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 1, 64)             21504     
                                                                 
 dropout_2 (Dropout)         (None, 1, 64)             0         
                                                                 
 lstm_4 (LSTM)               (None, 1, 64)             33024     
                                                                 
 dropout_3 (Dropout)         (None, 1, 64)             0         
                                                                 
 lstm_5 (LSTM)               (None, 1, 64)             33024     
                                                                 
 flatten_1 (Flatten)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 50)               

In [40]:
history = model.fit(X_train, y_train, epochs=100, batch_size=5000,validation_split=0.2)

Epoch 1/100


33/33 [==============================] - 7s 81ms/step - loss: 1.7192 - accuracy: 0.8123 - val_loss: 1.2204 - val_accuracy: 0.8324
Epoch 2/100
33/33 [==============================] - 2s 46ms/step - loss: 0.7973 - accuracy: 0.8308 - val_loss: 0.6005 - val_accuracy: 0.8324
Epoch 3/100
33/33 [==============================] - 2s 46ms/step - loss: 0.5558 - accuracy: 0.8308 - val_loss: 0.5089 - val_accuracy: 0.8324
Epoch 4/100
33/33 [==============================] - 2s 46ms/step - loss: 0.5006 - accuracy: 0.8332 - val_loss: 0.4764 - val_accuracy: 0.8381
Epoch 5/100
33/33 [==============================] - 2s 47ms/step - loss: 0.4654 - accuracy: 0.8473 - val_loss: 0.4272 - val_accuracy: 0.8784
Epoch 6/100
33/33 [==============================] - 2s 49ms/step - loss: 0.3901 - accuracy: 0.9074 - val_loss: 0.3417 - val_accuracy: 0.9230
Epoch 7/100
33/33 [==============================] - 2s 55ms/step - loss: 0.3360 - accuracy: 0.9192 - val_loss: 0.3125 - val_accuracy: 0.9221
Epoc